In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import DateType

spark = SparkSession.builder \
    .appName("PostgreSQL to ClickHouse Reports") \
    .config("spark.jars", "/home/jovyan/jars/postgresql-42.6.0.jar,/home/jovyan/jars/clickhouse-jdbc-0.4.6.jar") \
    .getOrCreate()

pg_url = "jdbc:postgresql://db:5432/mydatabase"
pg_properties = {
    "user": "user_bd",
    "password": "password_bd",
    "driver": "org.postgresql.Driver"
}

clickhouse_url = "jdbc:clickhouse://clickhouse:8123/default"
clickhouse_driver = "com.clickhouse.jdbc.ClickHouseDriver"
clickhouse_properties = {
    "user": "default",
    "password": "",
    "driver": clickhouse_driver
}

In [2]:
print("Чтение данных из PostgreSQL...")

fact_sales = spark.read.jdbc(url=pg_url, table="FactSales", properties=pg_properties)
dim_customers = spark.read.jdbc(url=pg_url, table="DimCustomers", properties=pg_properties)
dim_sellers = spark.read.jdbc(url=pg_url, table="DimSellers", properties=pg_properties)
dim_products = spark.read.jdbc(url=pg_url, table="DimProducts", properties=pg_properties)
dim_stores = spark.read.jdbc(url=pg_url, table="DimStores", properties=pg_properties)
dim_suppliers = spark.read.jdbc(url=pg_url, table="DimSuppliers", properties=pg_properties)
dim_date = spark.read.jdbc(url=pg_url, table="DimDate", properties=pg_properties)


Чтение данных из PostgreSQL...


# --- Отчеты по продуктам ---

In [3]:

print("\n--- Витрина продаж по продуктам ---")

print("Создание отчета: Топ-10 самых продаваемых продуктов...")
product_sales_top10_products = fact_sales.groupBy("product_sk") \
    .agg(
        F.sum("sale_quantity").alias("total_sold_units"),
        F.sum("sale_total_price").alias("total_revenue")
    ) \
    .join(dim_products, "product_sk") \
    .select(
        F.col("product_name"),
        F.col("product_category"),
        F.col("total_sold_units"),
        F.col("total_revenue")
    ) \
    .orderBy(F.desc("total_sold_units")) \
    .limit(10)

product_sales_top10_products.show()
create_table_sql_options = "ENGINE = MergeTree() ORDER BY (product_name, total_sold_units) PRIMARY KEY (product_name)"
product_sales_top10_products.write.format("jdbc") \
    .mode("overwrite") \
    .option("url", clickhouse_url) \
    .option("dbtable", "product_sales_top10_products") \
    .option("driver", clickhouse_driver) \
    .option("createTableOptions", create_table_sql_options) \
    .save()
print("Отчет 'product_sales_top10_products' сохранен.")


print("Создание отчета: Общая выручка по категориям продуктов...")
product_sales_revenue_by_category = fact_sales.join(dim_products, "product_sk") \
    .groupBy("product_category") \
    .agg(
        F.sum("sale_total_price").alias("total_category_revenue"),
        F.sum("sale_quantity").alias("total_category_sold_units")
    ) \
    .orderBy(F.desc("total_category_revenue"))

product_sales_revenue_by_category.show()
create_table_sql_options = "ENGINE = MergeTree() ORDER BY (product_category, total_category_revenue) PRIMARY KEY (product_category)"
product_sales_revenue_by_category.write.format("jdbc") \
    .mode("overwrite") \
    .option("url", clickhouse_url) \
    .option("dbtable", "product_sales_revenue_by_category") \
    .option("driver", clickhouse_driver) \
    .option("createTableOptions", create_table_sql_options) \
    .save()
print("Отчет 'product_sales_revenue_by_category' сохранен.")


print("Создание отчета: Средний рейтинг и количество отзывов для каждого продукта...")
product_sales_avg_rating_reviews = dim_products.select(
    F.col("product_name"),
    F.col("product_category"),
    F.col("product_rating"),
    F.col("product_reviews")
).orderBy(F.desc("product_rating"), F.desc("product_reviews"))

product_sales_avg_rating_reviews.show()
create_table_sql_options = "ENGINE = MergeTree() ORDER BY (product_name, product_rating, product_reviews) PRIMARY KEY (product_name)"
product_sales_avg_rating_reviews.write.format("jdbc") \
    .mode("overwrite") \
    .option("url", clickhouse_url) \
    .option("dbtable", "product_sales_avg_rating_reviews") \
    .option("driver", clickhouse_driver) \
    .option("createTableOptions", create_table_sql_options) \
    .save()
print("Отчет 'product_sales_avg_rating_reviews' сохранен.")



--- Витрина продаж по продуктам ---
Создание отчета: Топ-10 самых продаваемых продуктов...
+------------+----------------+----------------+-------------+
|product_name|product_category|total_sold_units|total_revenue|
+------------+----------------+----------------+-------------+
|    Dog Food|            Cage|           18298|    848567.19|
|   Bird Cage|            Food|           18205|    847478.05|
|     Cat Toy|            Cage|           18120|    833806.88|
+------------+----------------+----------------+-------------+

Отчет 'product_sales_top10_products' сохранен.
Создание отчета: Общая выручка по категориям продуктов...
+----------------+----------------------+-------------------------+
|product_category|total_category_revenue|total_category_sold_units|
+----------------+----------------------+-------------------------+
|            Cage|            1682374.07|                    36418|
|            Food|             847478.05|                    18205|
+----------------+---

# --- Отчеты по клиентам ---

In [4]:

print("\n--- Витрина продаж по клиентам ---")

print("Создание отчета: Топ-10 клиентов с наибольшей общей суммой покупок...")
customer_sales_top10_customers = fact_sales.groupBy("customer_sk") \
    .agg(
        F.sum("sale_total_price").alias("total_purchase_amount"),
        F.countDistinct("sale_id").alias("number_of_purchases")
    ) \
    .join(dim_customers, "customer_sk") \
    .select(
        F.col("customer_first_name"),
        F.col("customer_last_name"),
        F.col("customer_email"),
        F.col("total_purchase_amount"),
        F.col("number_of_purchases")
    ) \
    .orderBy(F.desc("total_purchase_amount")) \
    .limit(10)

customer_sales_top10_customers.show()
create_table_sql_options = "ENGINE = MergeTree() ORDER BY (customer_email, total_purchase_amount) PRIMARY KEY (customer_email)"
customer_sales_top10_customers.write.format("jdbc") \
    .mode("overwrite") \
    .option("url", clickhouse_url) \
    .option("dbtable", "customer_sales_top10_customers") \
    .option("driver", clickhouse_driver) \
    .option("createTableOptions", create_table_sql_options) \
    .save()
print("Отчет 'customer_sales_top10_customers' сохранен.")

print("Создание отчета: Распределение клиентов по странам...")
customer_sales_by_country = dim_customers.groupBy("customer_country") \
    .agg(
        F.count("customer_sk").alias("number_of_customers")
    ) \
    .orderBy(F.desc("number_of_customers"))

customer_sales_by_country.show()
create_table_sql_options = "ENGINE = MergeTree() ORDER BY (customer_country, number_of_customers) PRIMARY KEY (customer_country)"
customer_sales_by_country.write.format("jdbc") \
    .mode("overwrite") \
    .option("url", clickhouse_url) \
    .option("dbtable", "customer_sales_by_country") \
    .option("driver", clickhouse_driver) \
    .option("createTableOptions", create_table_sql_options) \
    .save()
print("Отчет 'customer_sales_by_country' сохранен.")

print("Создание отчета: Средний чек для каждого клиента...")
customer_sales_avg_receipt_per_customer = fact_sales.groupBy("customer_sk") \
    .agg(
        F.sum("sale_total_price").alias("total_revenue"),
        F.countDistinct("sale_id").alias("number_of_sales")
    ) \
    .withColumn("average_receipt", F.col("total_revenue") / F.col("number_of_sales")) \
    .join(dim_customers, "customer_sk") \
    .select(
        F.col("customer_first_name"),
        F.col("customer_last_name"),
        F.col("customer_email"),
        F.col("average_receipt")
    ) \
    .orderBy(F.desc("average_receipt"))

customer_sales_avg_receipt_per_customer.show()
create_table_sql_options = "ENGINE = MergeTree() ORDER BY (customer_email, average_receipt) PRIMARY KEY (customer_email)"
customer_sales_avg_receipt_per_customer.write.format("jdbc") \
    .mode("overwrite") \
    .option("url", clickhouse_url) \
    .option("dbtable", "customer_sales_avg_receipt") \
    .option("driver", clickhouse_driver) \
    .option("createTableOptions", create_table_sql_options) \
    .save()
print("Отчет 'customer_sales_avg_receipt' сохранен.")



--- Витрина продаж по клиентам ---
Создание отчета: Топ-10 клиентов с наибольшей общей суммой покупок...
+-------------------+------------------+--------------------+---------------------+-------------------+
|customer_first_name|customer_last_name|      customer_email|total_purchase_amount|number_of_purchases|
+-------------------+------------------+--------------------+---------------------+-------------------+
|                Gus|         Hartshorn| bfeasby57@youku.com|               499.85|                  1|
|              Hayes|            McKain|sstappardbp@busin...|               499.80|                  1|
|                Ava|             Lomas|dsorea0@geocities...|               499.76|                  1|
|              Dawna|             Impey|    rivattspm@un.org|               499.76|                  1|
|            Lavinia|         Horsburgh|previllh3@tinyurl...|               499.73|                  1|
|               Dame|        Auchinleck|jthurnhamqe@sourc...| 

# --- Отчеты по времени ---

In [5]:
print("\n--- Витрина продаж по времени ---")

print("Создание отчета: Месячные и годовые тренды продаж...")
time_sales_trends = fact_sales.join(dim_date, "date_sk") \
    .groupBy("year", "month", "month_name") \
    .agg(
        F.sum("sale_quantity").alias("monthly_sold_units"),
        F.sum("sale_total_price").alias("monthly_revenue")
    ) \
    .orderBy("year", "month")

time_sales_trends.show()
create_table_sql_options = "ENGINE = MergeTree() ORDER BY (year, month) PRIMARY KEY (year, month)"
time_sales_trends.write.format("jdbc") \
    .mode("overwrite") \
    .option("url", clickhouse_url) \
    .option("dbtable", "time_sales_trends") \
    .option("driver", clickhouse_driver) \
    .option("createTableOptions", create_table_sql_options) \
    .save()
print("Отчет 'time_sales_trends' сохранен.")


print("Создание отчета: Средний размер заказа по месяцам...")
time_sales_avg_order_size_monthly = fact_sales.join(dim_date, "date_sk") \
    .groupBy("year", "month", "month_name") \
    .agg(
        F.sum("sale_total_price").alias("total_monthly_revenue"),
        F.countDistinct("sale_id").alias("number_of_orders") # Assuming sale_id is unique per order
    ) \
    .withColumn("average_order_size", F.col("total_monthly_revenue") / F.col("number_of_orders")) \
    .orderBy("year", "month")

time_sales_avg_order_size_monthly.show()
create_table_sql_options = "ENGINE = MergeTree() ORDER BY (year, month) PRIMARY KEY (year, month)"
time_sales_avg_order_size_monthly.write.format("jdbc") \
    .mode("overwrite") \
    .option("url", clickhouse_url) \
    .option("dbtable", "time_sales_avg_order_size_monthly") \
    .option("driver", clickhouse_driver) \
    .option("createTableOptions", create_table_sql_options) \
    .save()
print("Отчет 'time_sales_avg_order_size_monthly' сохранен.")



--- Витрина продаж по времени ---
Создание отчета: Месячные и годовые тренды продаж...
+----+-----+----------+------------------+---------------+
|year|month|month_name|monthly_sold_units|monthly_revenue|
+----+-----+----------+------------------+---------------+
|2021|    1|   January|              4856|      224158.54|
|2021|    2|  February|              4070|      192348.31|
|2021|    3|     March|              4561|      207282.20|
|2021|    4|     April|              4564|      206592.82|
|2021|    5|       May|              4451|      211764.86|
|2021|    6|      June|              4438|      215042.80|
|2021|    7|      July|              4750|      220496.51|
|2021|    8|    August|              4818|      221275.78|
|2021|    9| September|              4507|      210623.43|
|2021|   10|   October|              4976|      228743.32|
|2021|   11|  November|              4297|      200154.69|
|2021|   12|  December|              4335|      191368.86|
+----+-----+----------+----

# --- Отчеты по магазинам ---

In [6]:
print("\n--- Витрина продаж по магазинам ---")

print("Создание отчета: Топ-5 магазинов с наибольшей выручкой...")
store_sales_top5_stores = fact_sales.groupBy("store_sk") \
    .agg(
        F.sum("sale_total_price").alias("total_store_revenue"),
        F.sum("sale_quantity").alias("total_store_sold_units")
    ) \
    .join(dim_stores, "store_sk") \
    .select(
        F.col("store_name"),
        F.col("store_city"),
        F.col("store_country"),
        F.col("total_store_revenue"),
        F.col("total_store_sold_units")
    ) \
    .orderBy(F.desc("total_store_revenue")) \
    .limit(5)

store_sales_top5_stores.show()
create_table_sql_options = "ENGINE = MergeTree() ORDER BY (store_name, total_store_revenue) PRIMARY KEY (store_name)"
store_sales_top5_stores.write.format("jdbc") \
    .mode("overwrite") \
    .option("url", clickhouse_url) \
    .option("dbtable", "store_sales_top5_stores") \
    .option("driver", clickhouse_driver) \
    .option("createTableOptions", create_table_sql_options) \
    .save()
print("Отчет 'store_sales_top5_stores' сохранен.")

print("Создание отчета: Распределение продаж по городам и странам...")
store_sales_by_location = fact_sales.join(dim_stores, "store_sk") \
    .groupBy("store_country", "store_city") \
    .agg(
        F.sum("sale_total_price").alias("total_sales_revenue"),
        F.sum("sale_quantity").alias("total_sales_quantity")
    ) \
    .orderBy(F.desc("total_sales_revenue"))

store_sales_by_location.show()
create_table_sql_options = "ENGINE = MergeTree() ORDER BY (store_country, store_city) PRIMARY KEY (store_country, store_city)"
store_sales_by_location.write.format("jdbc") \
    .mode("overwrite") \
    .option("url", clickhouse_url) \
    .option("dbtable", "store_sales_by_location") \
    .option("driver", clickhouse_driver) \
    .option("createTableOptions", create_table_sql_options) \
    .save()
print("Отчет 'store_sales_by_location' сохранен.")

print("Создание отчета: Средний чек для каждого магазина...")
store_sales_avg_receipt_per_store = fact_sales.groupBy("store_sk") \
    .agg(
        F.sum("sale_total_price").alias("total_revenue"),
        F.countDistinct("sale_id").alias("number_of_sales")
    ) \
    .withColumn("average_receipt", F.col("total_revenue") / F.col("number_of_sales")) \
    .join(dim_stores, "store_sk") \
    .select(
        F.col("store_name"),
        F.col("store_city"),
        F.col("average_receipt")
    ) \
    .orderBy(F.desc("average_receipt"))

store_sales_avg_receipt_per_store.show()
create_table_sql_options = "ENGINE = MergeTree() ORDER BY (store_name, average_receipt) PRIMARY KEY (store_name)"
store_sales_avg_receipt_per_store.write.format("jdbc") \
    .mode("overwrite") \
    .option("url", clickhouse_url) \
    .option("dbtable", "store_sales_avg_receipt") \
    .option("driver", clickhouse_driver) \
    .option("createTableOptions", create_table_sql_options) \
    .save()
print("Отчет 'store_sales_avg_receipt' сохранен.")



--- Витрина продаж по магазинам ---
Создание отчета: Топ-5 магазинов с наибольшей выручкой...
+----------+---------------+-------------+-------------------+----------------------+
|store_name|     store_city|store_country|total_store_revenue|total_store_sold_units|
+----------+---------------+-------------+-------------------+----------------------+
|     Mynte|        Brunflo|        China|           15751.71|                   317|
|     Quatz|        Gemuruh|  Philippines|           15176.64|                   342|
|      Jayo|Banjar Penyalin|        China|           13976.01|                   273|
|     Quinu|         Bélabo|       France|           13952.88|                   273|
|  Realcube|     Cincinnati|        China|           13700.77|                   274|
+----------+---------------+-------------+-------------------+----------------------+

Отчет 'store_sales_top5_stores' сохранен.
Создание отчета: Распределение продаж по городам и странам...
+--------------------+----

# --- Отчеты по поставщикам ---

In [7]:
print("\n--- Витрина продаж по поставщикам ---")

print("Создание отчета: Топ-5 поставщиков с наибольшей выручкой...")
supplier_sales_top5_suppliers = fact_sales.groupBy("supplier_sk") \
    .agg(
        F.sum("sale_total_price").alias("total_supplier_revenue"),
        F.sum("sale_quantity").alias("total_supplied_units")
    ) \
    .join(dim_suppliers, "supplier_sk") \
    .select(
        F.col("supplier_name"),
        F.col("supplier_country"),
        F.col("total_supplier_revenue"),
        F.col("total_supplied_units")
    ) \
    .orderBy(F.desc("total_supplier_revenue")) \
    .limit(5)

supplier_sales_top5_suppliers.show()
create_table_sql_options = "ENGINE = MergeTree() ORDER BY (supplier_name, total_supplier_revenue) PRIMARY KEY (supplier_name)"
supplier_sales_top5_suppliers.write.format("jdbc") \
    .mode("overwrite") \
    .option("url", clickhouse_url) \
    .option("dbtable", "supplier_sales_top5_suppliers") \
    .option("driver", clickhouse_driver) \
    .option("createTableOptions", create_table_sql_options) \
    .save()
print("Отчет 'supplier_sales_top5_suppliers' сохранен.")


print("Создание отчета: Средняя цена товаров от каждого поставщика...")
supplier_sales_avg_product_price = fact_sales.join(dim_products, "product_sk") \
    .join(dim_suppliers, "supplier_sk") \
    .groupBy("supplier_sk", "supplier_name") \
    .agg(
        F.avg("product_price").alias("average_product_price_from_supplier")
    ) \
    .orderBy(F.desc("average_product_price_from_supplier"))

supplier_sales_avg_product_price.show()
create_table_sql_options = "ENGINE = MergeTree() ORDER BY (supplier_name, average_product_price_from_supplier) PRIMARY KEY (supplier_name)"
supplier_sales_avg_product_price.write.format("jdbc") \
    .mode("overwrite") \
    .option("url", clickhouse_url) \
    .option("dbtable", "supplier_sales_avg_product_price") \
    .option("driver", clickhouse_driver) \
    .option("createTableOptions", create_table_sql_options) \
    .save()
print("Отчет 'supplier_sales_avg_product_price' сохранен.")

print("Создание отчета: Распределение продаж по странам поставщиков...")
supplier_sales_by_country = fact_sales.join(dim_suppliers, "supplier_sk") \
    .groupBy("supplier_country") \
    .agg(
        F.sum("sale_total_price").alias("total_sales_revenue"),
        F.sum("sale_quantity").alias("total_sales_quantity")
    ) \
    .orderBy(F.desc("total_sales_revenue"))

supplier_sales_by_country.show()
create_table_sql_options = "ENGINE = MergeTree() ORDER BY (supplier_country, total_sales_revenue) PRIMARY KEY (supplier_country)"
supplier_sales_by_country.write.format("jdbc") \
    .mode("overwrite") \
    .option("url", clickhouse_url) \
    .option("dbtable", "supplier_sales_by_country") \
    .option("driver", clickhouse_driver) \
    .option("createTableOptions", create_table_sql_options) \
    .save()
print("Отчет 'supplier_sales_by_country' сохранен.")



--- Витрина продаж по поставщикам ---
Создание отчета: Топ-5 поставщиков с наибольшей выручкой...
+-------------+----------------+----------------------+--------------------+
|supplier_name|supplier_country|total_supplier_revenue|total_supplied_units|
+-------------+----------------+----------------------+--------------------+
|       Wikizz|        Pakistan|              17729.73|                 351|
|     Livetube|       Indonesia|              17422.08|                 305|
|         Katz|          Russia|              16372.18|                 292|
|        Mynte|     Philippines|              14784.34|                 293|
|         Jayo|         Moldova|              14409.04|                 289|
+-------------+----------------+----------------------+--------------------+

Отчет 'supplier_sales_top5_suppliers' сохранен.
Создание отчета: Средняя цена товаров от каждого поставщика...
+-----------+-------------+-----------------------------------+
|supplier_sk|supplier_name|avera

# --- Отчеты по качеству продукции ---

In [8]:
print("\n--- Витрина качества продукции ---")

print("Создание отчета: Продукты с наивысшим и наименьшим рейтингом...")
product_quality_top_rated = dim_products.select(
    F.col("product_name"),
    F.col("product_category"),
    F.col("product_rating"),
    F.col("product_reviews")
).orderBy(F.desc("product_rating")).limit(10)

product_quality_top_rated.show()
create_table_sql_options = "ENGINE = MergeTree() ORDER BY (product_name, product_rating) PRIMARY KEY (product_name)"
product_quality_top_rated.write.format("jdbc") \
    .mode("overwrite") \
    .option("url", clickhouse_url) \
    .option("dbtable", "product_quality_top_rated") \
    .option("driver", clickhouse_driver) \
    .option("createTableOptions", create_table_sql_options) \
    .save()
print("Отчет 'product_quality_top_rated' сохранен.")

product_quality_lowest_rated = dim_products.select(
    F.col("product_name"),
    F.col("product_category"),
    F.col("product_rating"),
    F.col("product_reviews")
).orderBy(F.asc("product_rating")).limit(10)

product_quality_lowest_rated.show()
create_table_sql_options = "ENGINE = MergeTree() ORDER BY (product_name, product_rating) PRIMARY KEY (product_name)"
product_quality_lowest_rated.write.format("jdbc") \
    .mode("overwrite") \
    .option("url", clickhouse_url) \
    .option("dbtable", "product_quality_lowest_rated") \
    .option("driver", clickhouse_driver) \
    .option("createTableOptions", create_table_sql_options) \
    .save()
print("Отчет 'product_quality_lowest_rated' сохранен.")


print("Создание отчета: Данные для корреляции между рейтингом и объемом продаж...")
product_quality_sales_correlation = fact_sales.groupBy("product_sk") \
    .agg(
        F.sum("sale_quantity").alias("total_sold_units"),
        F.sum("sale_total_price").alias("total_revenue")
    ) \
    .join(dim_products, "product_sk") \
    .select(
        F.col("product_name"),
        F.col("product_rating"),
        F.col("total_sold_units"),
        F.col("total_revenue")
    )

product_quality_sales_correlation.show()
create_table_sql_options = "ENGINE = MergeTree() ORDER BY (product_name) PRIMARY KEY (product_name)"
product_quality_sales_correlation.write.format("jdbc") \
    .mode("overwrite") \
    .option("url", clickhouse_url) \
    .option("dbtable", "product_quality_sales_correlation") \
    .option("driver", clickhouse_driver) \
    .option("createTableOptions", create_table_sql_options) \
    .save()
print("Отчет 'product_quality_sales_correlation' сохранен.")

print("Создание отчета: Продукты с наибольшим количеством отзывов...")
product_quality_top_reviews = dim_products.select(
    F.col("product_name"),
    F.col("product_category"),
    F.col("product_reviews"),
    F.col("product_rating")
).orderBy(F.desc("product_reviews")).limit(10)

product_quality_top_reviews.show()
create_table_sql_options = "ENGINE = MergeTree() ORDER BY (product_name, product_reviews) PRIMARY KEY (product_name)"
product_quality_top_reviews.write.format("jdbc") \
    .mode("overwrite") \
    .option("url", clickhouse_url) \
    .option("dbtable", "product_quality_top_reviews") \
    .option("driver", clickhouse_driver) \
    .option("createTableOptions", create_table_sql_options) \
    .save()
print("Отчет 'product_quality_top_reviews' сохранен.")



--- Витрина качества продукции ---
Создание отчета: Продукты с наивысшим и наименьшим рейтингом...
+------------+----------------+--------------+---------------+
|product_name|product_category|product_rating|product_reviews|
+------------+----------------+--------------+---------------+
|   Bird Cage|            Food|           3.6|            360|
|     Cat Toy|            Cage|           3.6|            938|
|    Dog Food|            Cage|           2.1|            754|
+------------+----------------+--------------+---------------+

Отчет 'product_quality_top_rated' сохранен.
+------------+----------------+--------------+---------------+
|product_name|product_category|product_rating|product_reviews|
+------------+----------------+--------------+---------------+
|    Dog Food|            Cage|           2.1|            754|
|   Bird Cage|            Food|           3.6|            360|
|     Cat Toy|            Cage|           3.6|            938|
+------------+----------------+-----

In [9]:
spark.stop()
print("SparkSession успешно завершена.")

SparkSession успешно завершена.
